In [1]:
f = open("c++98-grammar.txt")

In [2]:
s = ''
k=0
for line in f.readlines():
    try:
        s+=line
        k+=1
    except Exception as e:
        print(k)
        raise e
print(s)
len(s)

=== A.1 Keywords [key] ===
#1 New context-dependent keywords are introduced into a program by typedef (7.1.3), namespace (7.3.1), class (clause 9), enumeration (7.2), and template (clause 14) declarations.
typedef-name:
	identifier
namespace-name:
	original-namespace-name
	namespace-alias
original-namespace-name:
	identifier
namespace-alias:
	identifier
class-name:
	identifier
	template-id
enum-name:
	identifier
template-name:
	identifier
#	Note that a typedef-name naming a class is also a class-name (9.1).
=== A.2 Lexical conventions [lex] ===
hex-quad:
	hexadecimal-digit hexadecimal-digit hexadecimal-digit hexadecimal-digit
universal-character-name:
	\u hex-quad
	\U hex-quad hex-quad
preprocessing-token:
	header-name
	identifier
	pp-number
	character-literal
	string-literal
	preprocessing-op-or-punc
	@
	$
	`
#	each non-white-space character that cannot be one of the above
token:
	identifier
	keyword
	literal
	operator
	punctuator
header-name:
	< h-char-sequence >
	" q-char-sequence "

18319

In [3]:
def is_letter(c):
    return c>='a' and c<='z' or c>='а' and c<='я' or c>='А' and c<='Я' 
def read_head(s,p):
    if p>=len(s): return None
    if s[p]!='=': return None
    #print(p,"head")
    key = ''
    key_start = False
    while p<len(s) and s[p]!='\n':
        if key_start:
            if s[p]==']':
                key_start = False
            else:
                key+=s[p]
        if not key_start and s[p]=='[':
            key_start = True
        p+=1
    return (p+1,key)
def read_comment(s,p):
    if p>=len(s): return None
    if s[p]!='#': return None
    #print(p,"comment")
    comm = ''
    while p<len(s) and s[p]!='\n':
        comm+=s[p]
        p+=1
    return (p+1,comm)
def read_node_head(s,p):
    if p>=len(s): return None
    if not is_letter(s[p]): return None
    #print(p,"node_head")
    name = ''
    end_name = False
    while p<len(s) and s[p]!='\n':
        if not end_name:
            if s[p]!=':':
                name+=s[p]
            else:
                end_name = True
                break
        p+=1
    else:
        raise Exception('":" not found - '+str(p))
    one_of = s[p:].startswith(": one of")
    while p<len(s) and s[p]!='\n':
        p+=1
    return (p+1,(name,one_of))
def read_node_line(s,p):
    if p>=len(s): return None
    if s[p]!='\t': return None
    #print(p,"node_line")
    p+=1
    line = ''
    while p<len(s) and s[p]!='\n':
        line +=s[p]
        p+=1
    return (p+1,line)
    

In [4]:
class Node:
    def __repr__(self):
        return repr(self.__dict__)
    def __str__(self):
        return str(self.__dict__)
d = {}

i=0
p=0
l=0
cur_node = None
cur_key = None
while True:
    l+=1
    r = read_head(s,p)
    if r!=None:
        p = r[0]
        cur_key = r[1]
        continue
    r = read_comment(s,p)
    if r!=None:
        p = r[0]
        if cur_node!=None:
            cur_node.comment.append(r[1])
        continue
    r = read_node_head(s,p)
    if r!=None:
        p = r[0]
        cur_node = Node()
        cur_name = r[1][0]
        if cur_name in d: print(cur_name)
        d[cur_name] = cur_node
        cur_node.one_of = r[1][1]
        cur_node.lines = []
        cur_node.key = cur_key
        cur_node.id = i
        cur_node.comment = []
        #print(i,cur_name)
        i+=1
        continue
    r = read_node_line(s,p)
    if r!=None:
        p=r[0]
        cur_node.lines.append(r[1])
        continue
    break
assert p>=len(s), (p,len(s),l)

typedef-name
enum-name
namespace-name
original-namespace-name
namespace-alias
class-name
template-name


In [5]:
# todo: products, links
# examples
# graph

In [6]:
import re
r = re.search(r'opt\b','noptr-abstract-declaratoropt parameters-and-qualifiers trailing-return-type')
r.start()

25

In [7]:
import re
def strip_opt(s):
    r = re.search(r'opt\b',s)
    if r==None:
        return [s]
    p1 = p2 = r.start()
    # 0:p1 - копируем, p1:p2 - копируем и не копируем, p2:p3 - не копируем, p3: - рекурсивно
    p3 = p2+3
    while p1>0 and s[p1]!=' ':
        p1-=1
    s1 = s[0:p1]
    s2 = s[0:p2]
    tail = strip_opt(s[p3:])
    r = []
    for ss in tail:
        r.append(s1+ss)
        r.append(s2+ss)
    return r

In [8]:
strip_opt('qw-er asdfopt')

['qw-er', 'qw-er asdf']

In [9]:
for k,v in d.items():
    if v.one_of:
        products = []
        for s in v.lines:
            cur_prod=''
            for c in s:
                if c==' ':
                    products.append([cur_prod,False])
                    cur_prod=''
                else:
                    cur_prod+=c
            if cur_prod!='':
                products.append([cur_prod,False])
        v.products = products
    else:
        products = []
        for s in v.lines:
            products+=[[x,False] for x in strip_opt(s)]
        v.products = products
        

In [10]:
keywords = {}
def make_links(s,key):
    global keywords
    in_link = False
    links = set()
    for p in range(len(s)):
        if not in_link:
            if (is_letter(s[p]) or s[p]=='-' or s[p]=='_' or s[p]>='0' and s[p]<='9'):
                in_link = True
                p1 = p
        else:
            if not (is_letter(s[p]) or s[p]=='-' or s[p]=='_' or s[p]>='0' and s[p]<='9'):
                in_link = False
                p2 = p
                if s[p1:p2] in d:
                    links.add(s[p1:p2])
                else:
                    if s[p1:p2] not in keywords:
                        keywords[s[p1:p2]] = set()
                    keywords[s[p1:p2]].add(key)
    if in_link:
        if s[p1:] in d:
            links.add(s[p1:])
        else:
            if s[p1:] not in keywords:
                keywords[s[p1:]] = set()
            keywords[s[p1:]].add(key)
    return links

In [11]:
for k,v in d.items():
    links = set()
    for pr in v.products:
        links |=make_links(pr[0],k)
    v.links = links
keywords

{'u': {'nondigit', 'universal-character-name', 'unsigned-suffix'},
 'preprocessing-op-or-punc': {'preprocessing-token'},
 'keyword': {'token'},
 'operator': {'conversion-function-id', 'operator-function-id', 'token'},
 'punctuator': {'token'},
 'a': {'c-char',
  'h-char',
  'hexadecimal-digit',
  'nondigit',
  'q-char',
  's-char',
  'simple-escape-sequence'},
 'b': {'c-char',
  'h-char',
  'hexadecimal-digit',
  'nondigit',
  'q-char',
  's-char',
  'simple-escape-sequence'},
 'c': {'c-char',
  'h-char',
  'hexadecimal-digit',
  'nondigit',
  'q-char',
  's-char'},
 'e': {'exponent-part', 'hexadecimal-digit', 'nondigit', 'pp-number'},
 'd': {'hexadecimal-digit', 'nondigit'},
 'f': {'floating-suffix',
  'hexadecimal-digit',
  'nondigit',
  'simple-escape-sequence'},
 'g': {'nondigit'},
 'h': {'nondigit'},
 'i': {'nondigit'},
 'j': {'nondigit'},
 'k': {'nondigit'},
 'l': {'floating-suffix', 'long-suffix', 'nondigit'},
 'm': {'nondigit'},
 'n': {'new-line', 'nondigit', 'simple-escape-seq

In [12]:
len(d)

201

In [13]:
x=0;
for k,v in d.items():
    x+=len(v.products)
x

817

In [14]:
import random
random.seed()

flag = True
counter = 0
while flag and counter <200:
    counter +=1
    print('ietr',counter)
    flag = False
    for k,v in d.items():
        for i in range(len(v.products)):
            if not v.products[i][1]:
                flag = True
                links =make_links(v.products[i][0],k)
                replaces = {}
                for l in links:
                    if hasattr(d[l],'examples'):
                        replaces[l] = d[l].examples[int(random.uniform(0,len(d[l].examples)))]
                    else: break
                if len(replaces)==len(links):
                    tmp = v.products[i][0]
                    for a,b in replaces.items():
                        tmp = re.sub(a,b,tmp)
                    if not hasattr(v,'examples'): 
                        v.examples = []
                    v.examples.append(tmp)
                    v.products[i][1] = True
                else:
                    print('miss',k,i,v.products[i][0])
flag

ietr 1
miss typedef-name 0 identifier
miss namespace-name 0 original-namespace-name
miss namespace-name 1 namespace-alias
miss original-namespace-name 0 identifier
miss namespace-alias 0 identifier
miss class-name 0 identifier
miss class-name 1 template-id
miss enum-name 0 identifier
miss template-name 0 identifier
miss hex-quad 0 hexadecimal-digit hexadecimal-digit hexadecimal-digit hexadecimal-digit
miss universal-character-name 0 \u hex-quad
miss universal-character-name 1 \U hex-quad hex-quad
miss preprocessing-token 0 header-name
miss preprocessing-token 1 identifier
miss preprocessing-token 2 pp-number
miss preprocessing-token 3 character-literal
miss preprocessing-token 4 string-literal
miss token 0 identifier
miss token 2 literal
miss header-name 0 < h-char-sequence >
miss header-name 1 " q-char-sequence "
miss h-char-sequence 0 h-char
miss h-char-sequence 1 h-char-sequence h-char
miss q-char-sequence 0 q-char
miss q-char-sequence 1 q-char-sequence q-char
miss pp-number 0 digit

miss if-group 3 # ifdef identifier new-line group
miss if-group 4 # ifndef identifier new-line
miss if-group 5 # ifndef identifier new-line group
miss elif-groups 0 elif-group
miss elif-groups 1 elif-groups elif-group
miss elif-group 0 # elif constant-expression new-line
miss elif-group 1 # elif constant-expression new-line group
miss else-group 0 # else new-line
miss else-group 1 # else new-line group
miss endif-line 0 # endif new-line
miss control-line 0 # include pp-tokens new-line
miss control-line 1 # define identifier replacement-list new-line
miss control-line 2 # define identifier lparen ) replacement-list new-line
miss control-line 3 # define identifier lparen identifier-list ) replacement-list new-line
miss control-line 4 # define identifier lparen ... ) replacement-list new-line
miss control-line 5 # define identifier lparen identifier-list, ... ) replacement-list new-line
miss control-line 6 # undef identifier new-line
miss control-line 7 # line pp-tokens new-line
miss cont

miss namespace-name 0 original-namespace-name
miss namespace-name 1 namespace-alias
miss original-namespace-name 0 identifier
miss namespace-alias 0 identifier
miss class-name 0 identifier
miss class-name 1 template-id
miss enum-name 0 identifier
miss template-name 0 identifier
miss preprocessing-token 0 header-name
miss preprocessing-token 1 identifier
miss preprocessing-token 3 character-literal
miss token 0 identifier
miss literal 1 character-literal
miss literal 2 floating-literal
miss translation-unit 1 declaration-seq
miss primary-expression 3 id-expression
miss unqualified-id 2 conversion-function-id
miss unqualified-id 3 ~ class-name
miss unqualified-id 4 template-id
miss qualified-id 0  nested-name-specifier unqualified-id
miss qualified-id 1 :: nested-name-specifier unqualified-id
miss qualified-id 2  nested-name-specifier template unqualified-id
miss qualified-id 3 :: nested-name-specifier template unqualified-id
miss qualified-id 6 :: template-id
miss nested-name-specifier 

miss block-declaration 4 using-directive
miss simple-declaration 2  init-declarator-list ;
miss simple-declaration 3 decl-specifier-seq init-declarator-list ;
miss type-specifier 4 typename-specifier
miss enum-specifier 2 enum{ enumerator-list}
miss enum-specifier 3 enum identifier{ enumerator-list}
miss namespace-alias-definition 0 namespace identifier = qualified-namespace-specifier ;
miss init-declarator-list 0 init-declarator
miss init-declarator-list 1 init-declarator-list , init-declarator
miss init-declarator 0 declarator
miss init-declarator 1 declarator initializer
miss function-definition 2  declarator ctor-initializer function-body
miss function-definition 3 decl-specifier-seq declarator ctor-initializer function-body
miss class-head 2 class-key base-clause
miss class-head 3 class-key identifier base-clause
miss class-head 5 class-key nested-name-specifier identifier base-clause
miss class-head 8 class-key template-id base-clause
miss class-head 9 class-key nested-name-speci

False

In [15]:
for k,v in d.items():
    if hasattr(v,'examples'): 
        for x in v.examples:
            print(k,x)
print('-------')
for k,v in d.items():
    if not hasattr(v,'examples'): 
        print(k, 'has no examples')

typedef-name \u 4 4 4 4 9
namespace-name \u 4 4 4 4 \u 4 4 4 4-nondigit
namespace-name \u 4 4 4 4 \u 4 4 4 4-nondigit
original-namespace-name \u 4 4 4 4 \u 4 4 4 4-nondigit
namespace-alias \u 4 4 4 4 \u 4 4 4 4-nondigit
class-name \u 4 4 4 4 \u 4 4 4 4-nondigit
class-name \u 4 4 4 4 9 < >
enum-name \u 4 4 4 4
template-name \u 4 4 4 4 9
hex-quad 4 4 4 4
universal-character-name \u 4 4 4 4
universal-character-name \U 4 4 4 4 4 4 4 4
preprocessing-token preprocessing-op-or-punc
preprocessing-token @
preprocessing-token $
preprocessing-token `
preprocessing-token "
preprocessing-token 2
preprocessing-token < c >
preprocessing-token \u 4 4 4 4 9
preprocessing-token ' b-sequence b '
token keyword
token operator
token punctuator
token false
token \u 4 4 4 4 \u 4 4 4 4-nondigit
header-name < c >
header-name " c "
h-char-sequence c
h-char-sequence b-sequence b
h-char a
h-char b
h-char c
h-char "
q-char-sequence c
q-char-sequence c c
q-char a
q-char b
q-char c
q-char >
pp-number 2
pp-number . 7


base-specifier-list virtual :: \u 4 4 4 4 \u 4 4 4 4-nondigit
base-specifier-list  \u 4 4 4 4 \u 4 4 4 4-nondigit-list ,  \u 4 4 4 4 \u 4 4 4 4-nondigit
base-specifier  \u 4 4 4 4 \u 4 4 4 4-nondigit
base-specifier :: \u 4 4 4 4 \u 4 4 4 4-nondigit
base-specifier virtual \u 4 4 4 4 \u 4 4 4 4-nondigit
base-specifier virtual private \u 4 4 4 4 \u 4 4 4 4-nondigit
base-specifier virtual :: \u 4 4 4 4 \u 4 4 4 4-nondigit
base-specifier virtual protected :: \u 4 4 4 4 \u 4 4 4 4-nondigit
base-specifier protected \u 4 4 4 4 \u 4 4 4 4-nondigit
base-specifier public virtual \u 4 4 4 4 \u 4 4 4 4-nondigit
base-specifier protected :: \u 4 4 4 4 \u 4 4 4 4-nondigit
base-specifier public virtual :: \u 4 4 4 4 \u 4 4 4 4-nondigit
base-specifier  \u 4 4 4 4 9 :: \u 4 4 4 4 9 < >
base-specifier :: \u 4 4 4 4 \u 4 4 4 4-nondigit :: template \u 4 4 4 4 9 < > :: \u 4 4 4 4 \u 4 4 4 4-nondigit
base-specifier virtual \u 4 4 4 4 \u 4 4 4 4-nondigit :: \u 4 4 4 4 \u 4 4 4 4-nondigit
base-specifier virtual

In [16]:
for k,v in d.items():
    for x in v.products:
        if not x[1]:
            print(k,':',x[0])

In [17]:
#todo добавить комментарии, граф .gml нарисовать, вручную написать примеры

In [18]:
code = """Creator	"yFiles"
Version	"2.12"
graph
[
	hierarchic	1
	label	""
	directed	1"""
for k,v in d.items():
    code+="\tnode\n\t[\n"
    code+="\t\tid\t"+str(v.id)+'\n'
    code+="\t\tlabel\t\""+k+':'
    code+=(' one of' if v.one_of else '')+'['+v.key+']\n'
    for l in v.lines:
        code+=l.replace('&',"&amp;").replace('"',"&quot;")+'\n'
    for l in v.comment:
        code+=l.replace('&',"&amp;").replace('"',"&quot;")+'\n'
    code+='-------\n'
    for l in v.products:
        code+='.\n'
    code+='"\n\t]\n'
for k,v in d.items():
    for l in v.links:
        code+='\tedge\n\t[\n\t\tsource\t'+str(v.id)+\
            '\n\t\ttarget\t'+str(d[l].id)+'\n'+\
"""		graphics
		[
			fill	"#000000"
			targetArrow	"standard"
		]
	]
"""
code+=']'
print(code)

Creator	"yFiles"
Version	"2.12"
graph
[
	hierarchic	1
	label	""
	directed	1	node
	[
		id	105
		label	"typedef-name:[dcl]
identifier
-------
.
"
	]
	node
	[
		id	115
		label	"namespace-name:[dcl]
original-namespace-name
namespace-alias
-------
.
.
"
	]
	node
	[
		id	116
		label	"original-namespace-name:[dcl]
identifier
-------
.
"
	]
	node
	[
		id	123
		label	"namespace-alias:[dcl]
identifier
-------
.
"
	]
	node
	[
		id	150
		label	"class-name:[class]
identifier
template-id
-------
.
.
"
	]
	node
	[
		id	110
		label	"enum-name:[dcl]
identifier
-------
.
"
	]
	node
	[
		id	177
		label	"template-name:[temp]
identifier
-------
.
"
	]
	node
	[
		id	7
		label	"hex-quad:[lex]
hexadecimal-digit hexadecimal-digit hexadecimal-digit hexadecimal-digit
-------
.
"
	]
	node
	[
		id	8
		label	"universal-character-name:[lex]
\u hex-quad
\U hex-quad hex-quad
-------
.
.
"
	]
	node
	[
		id	9
		label	"preprocessing-token:[lex]
header-name
identifier
pp-number
character-literal
string-literal
preprocessi

In [19]:
f = open("c++98-grammar.gml",'w')
f.write(code)
f.close()